In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [2]:
# Load the preprocessed data
with open("C:/Users/syafi/Desktop/py-1/ignore file/preprocessed_data.pkl", "rb") as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

with open("C:/Users/syafi/Desktop/py-1/ignore file/label_mapping.pkl", "rb") as f:
    label_mapping = pickle.load(f)
class_labels = list(label_mapping.keys())

# Display train and test size
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Train size: 1344719
Test size: 336180


In [3]:
# Define function to evaluate model
def evaluate_model(model, X_test_subset, y_test_subset, class_labels):
    predictions = model.predict(X_test_subset)
    accuracy = accuracy_score(y_test_subset, predictions)
    unique_classes = np.unique(y_test_subset)
    dynamic_labels = [class_labels[i] for i in unique_classes]
    report = classification_report(
        y_test_subset,
        predictions,
        target_names=dynamic_labels,
        labels=unique_classes,
        zero_division=0,
        output_dict=True,
    )
    return accuracy, report, predictions, unique_classes, dynamic_labels

In [4]:
# Flatten classification report
def flatten_classification_report(report, sample_size, train_size, test_size, accuracy):
    flat_report = {
        f"{label}_{metric}": value
        for label, metrics in report.items()
        if isinstance(metrics, dict)
        for metric, value in metrics.items()
    }
    flat_report.update(
        {
            "sample_size": sample_size,
            "train_size": train_size,
            "test_size": test_size,
            "accuracy": accuracy,
        }
    )
    return flat_report

In [5]:
# Expand the test set for better results
def expand_test_set(X_test, y_test, repeat_factor):
    X_test_expanded = np.repeat(X_test, repeats=repeat_factor, axis=0)
    y_test_expanded = np.repeat(y_test, repeats=repeat_factor)
    return X_test_expanded, y_test_expanded

In [6]:
# Define experiment sizes
sample_sizes = [25, 50, 75, 100, 250, 500, 750, 1000, 2500, 5000, 7500, 10000, 20000, 30000, 40000, 50000]
training_sizes = [20, 40, 60, 80, 200, 400, 600, 800, 2000, 4000, 6000, 8000, 16000, 24000, 32000, 40000]
testing_sizes = [5, 10, 15, 20, 50, 100, 150, 200, 500, 1000, 1500, 2000, 4000, 6000, 8000, 10000]

# Container for results
results = []
best_accuracy = 0
best_model = None
best_sample_size = 0

In [7]:
# Iterate through sample, training, and testing sizes
for sample_size, train_size, test_size in zip(
    sample_sizes, training_sizes, testing_sizes
):
    # Select subset of training and testing data
    X_train_subset, y_train_subset = X_train[:train_size], y_train[:train_size]
    X_test_subset, y_test_subset = X_test[:test_size], y_test[:test_size]

    # Expand the test set for alignment
    repeat_factor = 11
    X_test_expanded, y_test_expanded = expand_test_set(
        X_test_subset, y_test_subset, repeat_factor
    )

    # Fit the AdaBoost model
    ada_model = AdaBoostClassifier(n_estimators=50, random_state=42)
    ada_model.fit(X_train_subset, y_train_subset)

    # Evaluate the model
    accuracy, report, predictions, unique_classes, dynamic_labels = evaluate_model(
        ada_model, X_test_expanded, y_test_expanded, class_labels
    )

    # Print results for the current iteration
    print(f"\nAdaBoost Sample size {sample_size} - Accuracy: {accuracy:.4f}")
    print(
        classification_report(
            y_test_expanded,
            predictions,
            target_names=dynamic_labels,
            labels=unique_classes,
            zero_division=0,
        )
    )

    # Flatten report and store results
    flat_report = flatten_classification_report(
        report, sample_size, train_size, test_size, accuracy
    )
    results.append(flat_report)

    # Update the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = ada_model
        best_sample_size = sample_size
        print(
            f"New best model found for sample size {sample_size} with accuracy {accuracy:.4f}"
        )

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost Sample size 25 - Accuracy: 0.0000
                                      precision    recall  f1-score   support

               Autonomic Dysfunction       0.00      0.00      0.00      11.0
      Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00      11.0
                             Healthy       0.00      0.00      0.00      11.0
                        Hypertension       0.00      0.00      0.00      11.0
Respiratory Disease (COPD or Asthma)       0.00      0.00      0.00      11.0

                           micro avg       0.00      0.00      0.00      55.0
                           macro avg       0.00      0.00      0.00      55.0
                        weighted avg       0.00      0.00      0.00      55.0


AdaBoost Sample size 50 - Accuracy: 0.2000
                                      precision    recall  f1-score   support

                         Arrhythmias       0.00      0.00      0.00        11
               Autonomic Dysfunction       0.00   

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00        33
                         Arrhythmias       0.05      1.00      0.10        11
               Autonomic Dysfunction       0.00      0.00      0.00        22
      Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00        22
                            Diabetes       0.00      0.00      0.00        22
                             Healthy       0.00      0.00      0.00        33
                        Hypertension       1.00      1.00      1.00        11
Respiratory Disease (COPD or Asthma)       0.00      0.00      0.00        44
            Stress-related Disorders       0.00      0.00      0.00        22

                            accuracy                           0.10       220
                           macro avg       0.12      0.22      0.12       220
                        weighted avg       0.05      0.10     

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


...


AdaBoost Sample size 30000 - Accuracy: 0.2700
                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00      5731
                         Arrhythmias       0.00      0.00      0.00      6226
                     Atherosclerosis       0.87      0.59      0.71      5753
               Autonomic Dysfunction       0.00      0.00      0.00      5962
        Cardiovascular Disease (CVD)       0.78      0.92      0.84      6204
      Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00      6039
                            Diabetes       0.00      0.00      0.00      5698
                             Healthy       0.00      0.00      0.00      6358
                        Hypertension       0.28      0.43      0.34      6171
Respiratory Disease (COPD or Asthma)       0.00      0.00      0.00      5797
            Stress-related Disorders       0.13      1.00      0.24      6061

               

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost Sample size 40000 - Accuracy: 0.2721
                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00      8019
                         Arrhythmias       0.00      0.00      0.00      8217
                     Atherosclerosis       0.88      0.63      0.73      7986
               Autonomic Dysfunction       0.00      0.00      0.00      8052
        Cardiovascular Disease (CVD)       0.78      0.92      0.85      8283
      Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00      8052
                            Diabetes       0.00      0.00      0.00      7711
                             Healthy       0.00      0.00      0.00      8162
                        Hypertension       0.28      0.43      0.34      8063
Respiratory Disease (COPD or Asthma)       0.00      0.00      0.00      7568
            Stress-related Disorders       0.13      1.00      0.23      7887

               

c:\Users\syafi\anaconda3\envs\mydatascienv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



AdaBoost Sample size 50000 - Accuracy: 0.2721
                                      precision    recall  f1-score   support

                             Anaemia       0.00      0.00      0.00      9856
                         Arrhythmias       0.00      0.00      0.00     10296
                     Atherosclerosis       0.88      0.63      0.74     10109
               Autonomic Dysfunction       0.00      0.00      0.00     10329
        Cardiovascular Disease (CVD)       0.78      0.92      0.84     10153
      Chronic Fatigue Syndrome (CFS)       0.00      0.00      0.00      9966
                            Diabetes       0.00      0.00      0.00      9625
                             Healthy       0.00      0.00      0.00     10197
                        Hypertension       0.27      0.42      0.33      9889
Respiratory Disease (COPD or Asthma)       0.00      0.00      0.00      9537
            Stress-related Disorders       0.13      1.00      0.24     10043

               

In [ ]:
# Save the best model
if best_model is not None:
    best_model_file = f"best_adaboost_model_sample_size_{best_sample_size}.pkl"
    with open(best_model_file, "wb") as model_file:
        pickle.dump(best_model, model_file)
    print(f"\nBest model saved as {best_model_file} with accuracy {best_accuracy:.4f}")

In [ ]:
# Save results to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("adaboost_results.csv", index=False)
print("\nResults saved to 'adaboost_results.csv'")

In [ ]:
# Display train and test set details
print(f"Train size: {len(X_train)}")
print(f"Test size: {len(X_test)}")
print("\nClass Distribution in Test Set:")
print(pd.Series(y_test).value_counts())